In [31]:
# Import of librairies
from geopy.distance import geodesic
import pandas as pd
import numpy as np # Not always necessary
import matplotlib.pyplot as plt # Not always necessary
import seaborn as sns # Not always necessary
from sklearn.model_selection import train_test_split # libraie énorme donc j'importe que les fonctions utiles
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.tree import plot_tree

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import( OneHotEncoder, StandardScaler, LabelEncoder )

from sklearn.metrics import( accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, 
                             f1_score, mean_squared_error, r2_score, RocCurveDisplay )

# Import of dataset + format datetime

In [32]:
df_fraud_detection = pd.read_csv("fraudTest.csv", index_col=0, parse_dates=[1])
pd.set_option('display.max_columns', None) # affiche toutes les colonnes
df_fraud_detection.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,11710,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,32780,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,49632,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# Datetime format for dob

In [72]:
df_fraud_detection['dob']= pd.to_datetime(df_fraud_detection['dob'])

KeyError: 'dob'

# Add age column

In [34]:
df_fraud_detection['age'] = (df_fraud_detection['trans_date_trans_time'] - df_fraud_detection['dob']).astype('<m8[Y]')
df_fraud_detection['age'] = df_fraud_detection['age'].astype(int)
# df_fraud_detection.info() # checking

# Add street_name column

In [35]:
df_fraud_detection['street_name'] = [ x.split(" ") for x in df_fraud_detection['street'] ]
df_fraud_detection['street_name'] = [ ' '.join(x[1:3]) for x in df_fraud_detection['street_name'] ]

# Add "time" columns

In [36]:
df_fraud_detection["hour"] = df_fraud_detection['trans_date_trans_time'].dt.hour.astype(int)
df_fraud_detection['dayofweek'] = df_fraud_detection['trans_date_trans_time'].dt.dayofweek.astype(int)
df_fraud_detection['month'] = df_fraud_detection['trans_date_trans_time'].dt.month.astype(int)
df_fraud_detection['year'] = df_fraud_detection['trans_date_trans_time'].dt.year.astype(int)
df_fraud_detection['dayofyear'] = df_fraud_detection['trans_date_trans_time'].dt.dayofyear.astype(int)
df_fraud_detection['dayofmonth'] = df_fraud_detection['trans_date_trans_time'].dt.day.astype(int)
df_fraud_detection['weekofyear'] = df_fraud_detection['trans_date_trans_time'].dt.weekofyear.astype(int)

C:\Users\edood\AppData\Local\Temp\ipykernel_11072\4202031866.py:7: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



# Add card issuer

In [37]:
# https://www.forbes.com/advisor/credit-cards/what-does-your-credit-card-number-mean/
df_fraud_detection['card_issuer_MMI'] = [ f'mmi{str(x)[0:1]}' for x in df_fraud_detection['cc_num']]
df_fraud_detection['card_issuer_Bank'] = [ int(str(x)[1:6]) for x in df_fraud_detection['cc_num']]
print(len(df_fraud_detection['card_issuer_MMI'].value_counts()))
print(len(df_fraud_detection['card_issuer_Bank'].value_counts()))

6
901


# Add merchant_ column

In [38]:
df_fraud_detection["merchant_"] = [ x.replace('fraud_', '') for x in df_fraud_detection["merchant"] ]

# Add column geometry for position and merch_position

In [39]:
df_fraud_detection['geometry'] = list(zip(df_fraud_detection['lat'], df_fraud_detection['long']))
df_fraud_detection['merch_geometry'] = list(zip(df_fraud_detection['merch_lat'], df_fraud_detection['merch_long']))

In [40]:
def distance(row):
    try: 
        return (geodesic(row['geometry'], row['merch_geometry']).km) 
    except:
        return np.nan

df_fraud_detection['distance'] = df_fraud_detection.apply(lambda row: distance(row), axis = 1 )


In [41]:
print(df_fraud_detection.shape)
display(df_fraud_detection.head())

(555719, 37)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,street_name,hour,dayofweek,month,year,dayofyear,dayofmonth,weekofyear,card_issuer_MMI,card_issuer_Bank,merchant_,geometry,merch_geometry,distance
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0,52,Darlene Green,12,6,6,2020,173,21,25,mmi2,29116,Kirlin and Sons,"(33.9659, -80.9355)","(33.986391, -81.200714)",24.613746
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0,30,Marsh Union,12,6,6,2020,173,21,25,mmi3,57303,Sporer-Keebler,"(40.3207, -110.436)","(39.450498, -109.960431)",104.834043
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,11710,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0,49,Valentine Point,12,6,6,2020,173,21,25,mmi3,59821,"Swaniawski, Nitzsche and Welch","(40.6729, -73.5365)","(40.49581, -74.196111)",59.204796
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,32780,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0,32,Krystal Mill,12,6,6,2020,173,21,25,mmi3,59191,Haley Group,"(28.5697, -80.8191)","(28.812398, -80.883061)",27.615117
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,49632,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0,64,Evan Roads,12,6,6,2020,173,21,25,mmi3,52682,Johnston-Casper,"(44.2529, -85.01700000000001)","(44.959148, -85.884734)",104.423175


In [42]:
df_fraud_detection['zip'] = df_fraud_detection['zip'].astype(str)
df_fraud_detection['distance'] = (df_fraud_detection['distance']).astype(float,2)
df_fraud_detection['amt'] = df_fraud_detection['amt'].astype(float,2)
df_fraud_detection['unix_time'] = df_fraud_detection['unix_time'] .astype(int)
df_fraud_detection.head()


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,street_name,hour,dayofweek,month,year,dayofyear,dayofmonth,weekofyear,card_issuer_MMI,card_issuer_Bank,merchant_,geometry,merch_geometry,distance
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0,52,Darlene Green,12,6,6,2020,173,21,25,mmi2,29116,Kirlin and Sons,"(33.9659, -80.9355)","(33.986391, -81.200714)",24.613746
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0,30,Marsh Union,12,6,6,2020,173,21,25,mmi3,57303,Sporer-Keebler,"(40.3207, -110.436)","(39.450498, -109.960431)",104.834043
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,11710,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0,49,Valentine Point,12,6,6,2020,173,21,25,mmi3,59821,"Swaniawski, Nitzsche and Welch","(40.6729, -73.5365)","(40.49581, -74.196111)",59.204796
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,32780,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0,32,Krystal Mill,12,6,6,2020,173,21,25,mmi3,59191,Haley Group,"(28.5697, -80.8191)","(28.812398, -80.883061)",27.615117
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,49632,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0,64,Evan Roads,12,6,6,2020,173,21,25,mmi3,52682,Johnston-Casper,"(44.2529, -85.01700000000001)","(44.959148, -85.884734)",104.423175


In [43]:
type(df_fraud_detection['amt'][0])

numpy.float64

In [44]:
df_fraud_detection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555719 entries, 0 to 555718
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   trans_date_trans_time  555719 non-null  datetime64[ns]
 1   cc_num                 555719 non-null  int64         
 2   merchant               555719 non-null  object        
 3   category               555719 non-null  object        
 4   amt                    555719 non-null  float64       
 5   first                  555719 non-null  object        
 6   last                   555719 non-null  object        
 7   gender                 555719 non-null  object        
 8   street                 555719 non-null  object        
 9   city                   555719 non-null  object        
 10  state                  555719 non-null  object        
 11  zip                    555719 non-null  object        
 12  lat                    555719 non-null  floa

In [45]:
# Delete duplicate columns
df_fraud_detection_with_deleted = df_fraud_detection.drop(columns=['street', 'dob', 'merchant', 'lat', 'long', 'merch_lat', 'merch_long']) # street => street_name, dob ==> age, merchant ==> merchant_
df_fraud_detection_with_deleted.head() # checking

,trans_date_trans_time,cc_num,category,amt,first,last,gender,city,state,zip,city_pop,job,trans_num,unix_time,is_fraud,age,street_name,hour,dayofweek,month,year,dayofyear,dayofmonth,weekofyear,card_issuer_MMI,card_issuer_Bank,merchant_,geometry,merch_geometry,distance
0,2020-06-21 12:14:25,2291163933867244,personal_care,2.86,Jeff,Elliott,M,Columbia,SC,29209,333497,Mechanical engineer,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,0,52,Darlene Green,12,6,6,2020,173,21,25,mmi2,29116,Kirlin and Sons,"(33.9659, -80.9355)","(33.986391, -81.200714)",24.613746
1,2020-06-21 12:14:33,3573030041201292,personal_care,29.84,Joanne,Williams,F,Altonah,UT,84002,302,"Sales professional, IT",324cc204407e99f51b0d6ca0055005e7,1371816873,0,30,Marsh Union,12,6,6,2020,173,21,25,mmi3,57303,Sporer-Keebler,"(40.3207, -110.436)","(39.450498, -109.960431)",104.834043
2,2020-06-21 12:14:53,3598215285024754,health_fitness,41.28,Ashley,Lopez,F,Bellmore,NY,11710,34496,"Librarian, public",c81755dbbbea9d5c77f094348a7579be,1371816893,0,49,Valentine Point,12,6,6,2020,173,21,25,mmi3,59821,"Swaniawski, Nitzsche and Welch","(40.6729, -73.5365)","(40.49581, -74.196111)",59.204796
3,2020-06-21 12:15:15,3591919803438423,misc_pos,60.05,Brian,Williams,M,Titusville,FL,32780,54767,Set designer,2159175b9efe66dc301f149d3d5abf8c,1371816915,0,32,Krystal Mill,12,6,6,2020,173,21,25,mmi3,59191,Haley Group,"(28.5697, -80.8191)","(28.812398, -80.883061)",27.615117
4,2020-06-21 12:15:17,3526826139003047,travel,3.19,Nathan,Massey,M,Falmouth,MI,49632,1126,Furniture designer,57ff021bd3f328f8738bb535c302a31b,1371816917,0,64,Evan Roads,12,6,6,2020,173,21,25,mmi3,52682,Johnston-Casper,"(44.2529, -85.01700000000001)","(44.959148, -85.884734)",104.423175


In [46]:
df_fraud_detection_with_deleted.isnull().any()

trans_date_trans_time    False
cc_num                   False
category                 False
amt                      False
first                    False
last                     False
gender                   False
city                     False
state                    False
zip                      False
city_pop                 False
job                      False
trans_num                False
unix_time                False
is_fraud                 False
age                      False
street_name              False
hour                     False
dayofweek                False
month                    False
year                     False
dayofyear                False
dayofmonth               False
weekofyear               False
card_issuer_MMI          False
card_issuer_Bank         False
merchant_                False
geometry                 False
merch_geometry           False
distance                 False
dtype: bool

In [47]:
df_fraud_detection.describe(include='all')

C:\Users\edood\AppData\Local\Temp\ipykernel_11072\1719009962.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

C:\Users\edood\AppData\Local\Temp\ipykernel_11072\1719009962.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,street_name,hour,dayofweek,month,year,dayofyear,dayofmonth,weekofyear,card_issuer_MMI,card_issuer_Bank,merchant_,geometry,merch_geometry,distance
count,555719,5.557190e+05,555719,555719,555719.000000,555719,555719,555719,555719,555719,555719,555719,555719.000000,555719.000000,5.557190e+05,555719,555719,555719,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,555719,555719.000000,555719.000000,555719.000000,555719.0,555719.000000,555719.000000,555719.000000,555719,555719.000000,555719,555719,555719,555719.000000
unique,544760,NaN,693,14,NaN,341,471,2,924,849,50,912,NaN,NaN,NaN,478,910,555719,NaN,NaN,NaN,NaN,NaN,919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,693,912,555719,NaN
top,2020-12-19 16:02:22,NaN,fraud_Kilback LLC,gas_transport,NaN,Christopher,Smith,F,444 Robert Mews,Birmingham,TX,82514,NaN,NaN,NaN,Film/video editor,1977-03-23 00:00:00,2da90c7d74bd46a0caf3777415b3ebd3,NaN,NaN,NaN,NaN,NaN,Andrea Glen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mmi3,NaN,Kilback LLC,"(43.0048, -108.8964)","(33.986391, -81.200714)",NaN
freq,4,NaN,1859,56370,NaN,11443,12146,304886,1474,2423,40393,1589,NaN,NaN,NaN,4119,2408,1,NaN,NaN,NaN,NaN,NaN,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209033,NaN,1859,1589,1,NaN
first,2020-06-21 12:14:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1924-10-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-12-31 23:59:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-01-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,4.178387e+17,NaN,NaN,69.392810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.543253,-90.231325,8.822189e+04,NaN,NaN,NaN,1.380679e+09,38.542798,-90.231380,0.003860,46.392571,NaN,12.809062,2.726779,9.508536,2020.0,275.524918,16.463904,40.114020,NaN,46748.816708,NaN,NaN,NaN,76.102742
std,NaN,1.309837e+18,NaN,NaN,156.745941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.061336,13.721780,3.003909e+05,NaN,NaN,NaN,5.201104e+06,5.095829,13.733071,0.062008,17.432428,NaN,6.810924,2.178681,1.978205,0.0,60.198729,8.955311,8.608557,NaN,27790.209299,NaN,NaN,NaN,29.092829
min,NaN,6.041621e+10,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.027100,-165.672300,2.300000e+01,NaN,NaN,NaN,1.371817e+09,19.027422,-166.671575,0.000000,15.000000,NaN,0.000000,0.000000,6.000000,2020.0,173.000000,1.000000,25.000000,NaN,114.000000,NaN,NaN,NaN,0.124180
25%,NaN,1.800429e+14,NaN,NaN,9.630000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.668900,-96.798000,7.410000e+02,NaN,NaN,NaN,1.376029e+09,34.755302,-96.905129,0.000000,33.000000,NaN,7.000000,1.000000,8.000000,2020.0,222.000000,9.000000,32.000000,NaN,23118.000000,NaN,NaN,NaN,55.307207


In [48]:
df_fraud_detection = df_fraud_detection.drop(columns=['trans_date_trans_time','merchant','first','last','street','city','state','zip','job','dob','trans_num','unix_time','merchant_','geometry','merch_geometry','cc_num','street_name','year'])

In [49]:
# % valeurs manquantes
# 100*df_fraud_detection.isnull().sum()/df_fraud_detection.shape[0]
display(df_fraud_detection.head(1))
df_fraud_detection.dtypes

,category,amt,gender,lat,long,city_pop,merch_lat,merch_long,is_fraud,age,hour,dayofweek,month,dayofyear,dayofmonth,weekofyear,card_issuer_MMI,card_issuer_Bank,distance
0,personal_care,2.86,M,33.9659,-80.9355,333497,33.986391,-81.200714,0,52,12,6,6,173,21,25,mmi2,29116,24.613746


category             object
amt                 float64
gender               object
lat                 float64
long                float64
city_pop              int64
merch_lat           float64
merch_long          float64
is_fraud              int64
age                   int32
hour                  int32
dayofweek             int32
month                 int32
dayofyear             int32
dayofmonth            int32
weekofyear            int32
card_issuer_MMI      object
card_issuer_Bank      int64
distance            float64
dtype: object

## Graph de correlation 

In [70]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

corr_matrix = round(df_fraud_detection.corr(), 1)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.values.tolist(),
                                  y = corr_matrix.index.values.tolist(),
                                  )
fig.update_layout(
    title = go.layout.Title(text = 'Correlation matrix', x = 0.5), showlegend = False, 
            autosize=False, width=700, height=500, margin=dict(l=150, r=150, t=80, b=20)),

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Séparation Variables explicative X et variable Cible/Target Y 

In [55]:
# Separate target variable Y from features X
target_name = "is_fraud"

Y = df_fraud_detection[:][target_name]
X = df_fraud_detection.drop(columns= [target_name])

## Séparation en Entrainement & Test 

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Préprocessing Pipeline

In [57]:
# Create pipeline for numeric features
numeric_features = X.select_dtypes([np.number]).columns # Automatically detect positions of numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = X.select_dtypes("object").columns # Automatically detect positions of categorical columns
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train) # Preprocessing influenceur
X_test = preprocessor.transform(X_test) # Preprocessing copieur

print(f'numeric_features : {numeric_features}')
print(f'categorical_features : {categorical_features}')

numeric_features : Index(['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'age',
       'hour', 'dayofweek', 'month', 'dayofyear', 'dayofmonth', 'weekofyear',
       'card_issuer_Bank', 'distance'],
      dtype='object')
categorical_features : Index(['category', 'gender', 'card_issuer_MMI'], dtype='object')


## Training model
### Qualitatif (Classification)

In [58]:
model = LogisticRegression() # Train mode

## Training

In [71]:
model.fit(X_train, Y_train)

c:\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

## Predictions

In [60]:
Y_train_pred = model.predict(X_train) # Predictions on training set
Y_test_pred = model.predict(X_test) # Prédictions on test set 

## Performances evaluation
### Qualitatif (Classification)

In [61]:
print("Accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("Accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))

Accuracy on training set :  0.995917449249283
Accuracy on test set :  0.995744259699129


## Save Model

In [65]:
# save
import joblib
joblib.dump(model, "model.joblib", compress=3)

['model.joblib']

In [69]:
df_fraud_detection.to_csv(r'formatedDataset.csv', index=False)